# Set ENV Variable to Project Path

In [11]:
# Automatically reload modules when they change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Insert project root folder in environment variable

In [12]:
import os
import sys

def find_project_root(start_path=None, markers=(".git", "pyproject.toml", "requirements.txt")):
    """
    Walks up from start_path until it finds one of the marker files/folders.
    Returns the path of the project root.
    """
    if start_path is None:
        start_path = os.getcwd()

    current_path = os.path.abspath(start_path)

    while True:
        # check if any marker exists in current path
        if any(os.path.exists(os.path.join(current_path, marker)) for marker in markers):
            return current_path

        new_path = os.path.dirname(current_path)  # parent folder
        if new_path == current_path:  # reached root of filesystem
            raise FileNotFoundError(f"None of the markers {markers} found above {start_path}")
        current_path = new_path

project_root = find_project_root()
print("Project root:", project_root)

if project_root not in sys.path:
    sys.path.insert(0, project_root)


Project root: c:\ds_analytics_projects\darshil_course\apache-pyspark\darshil-pyspark


# Import Libraries

Import packages

In [13]:
import pandas as pd
import numpy as np
from pathlib import Path

Relative import

In [14]:
from utils.file_utils import get_project_path

In [15]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
    .appName("sparkSQL") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

# 📒 Spark SQL

---

### 🔎 Step 1: What is Spark SQL?

- **Spark SQL** lets you run SQL queries on top of Spark DataFrames and tables.
- You can use:
    - **ANSI-SQL** (standard SQL syntax).
    - **HiveQL** (Hive-compatible syntax).
- Queries run on Spark's distributed engine and can use **system functions, UDFs, and optimizations**.

👉 Before Spark, Hive was the de facto SQL layer for big data. Spark SQL replaced Hive in many systems because:

- It's faster,
- It integrates directly with DataFrames,
- It can connect to Hive metastores for compatibility.

---

### 🔎 Step 2: Running SQL Queries in Spark

**Simple SQL command:**

In [16]:
spark.sql("SELECT 1 + 1").show()

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



**Register DataFrame as a SQL view:**

In [17]:
json_path = str(Path(get_project_path('data', 'darshil-data', 'flight-data', 'json', '2015-summary.json')))

df = spark.read.json(json_path)
df.createOrReplaceTempView("flights_view")

result = spark.sql("""
SELECT DEST_COUNTRY_NAME, SUM(count) as total_count
FROM flights_view
GROUP BY DEST_COUNTRY_NAME
""")

result.where("DEST_COUNTRY_NAME LIKE 'S%'").where("total_count > 10").show()

+--------------------+-----------+
|   DEST_COUNTRY_NAME|total_count|
+--------------------+-----------+
|             Senegal|         40|
|              Sweden|        118|
|               Spain|        420|
|    Saint Barthelemy|         39|
|Saint Kitts and N...|        139|
|         South Korea|       1048|
|        Sint Maarten|        325|
|        Saudi Arabia|         83|
|         Switzerland|        294|
|         Saint Lucia|        123|
|               Samoa|         25|
|        South Africa|         36|
+--------------------+-----------+



👉 **Note:**

- `createOrReplaceTempView` makes the DataFrame queryable via SQL.
- You can then mix SQL and DataFrame API (`where`, `select`, etc.).

---

### 🔎 Step 3: Spark Tables

- **Table = Persistent data structure** managed by Spark.
- **DataFrame = Temporary, in code**.
- Key difference:
    - DataFrame exists only in the program.
    - Table lives inside a **database** (metadata tracked by Spark/Hive metastore).

---

### 🔎 Step 4: Managed vs Unmanaged Tables

- **Managed Table:**
    - Created with `saveAsTable`.
    - Spark manages both **data and metadata**.
    - Dropping table deletes the data.
- **Unmanaged Table (External):**
    - Created from files on disk (`CREATE EXTERNAL TABLE`).
    - Spark manages **metadata only**.
    - Dropping table does **not** delete files.

---

### 🔎 Step 5: Creating Tables

**Create table directly from files:**

In [18]:
# Convert to file URI
json_path_uri = Path(json_path).as_uri()
print(f"URI path: '{json_path_uri}'")

spark.sql("DROP TABLE IF EXISTS flights")

spark.sql(f"""
CREATE TABLE flights (
    DEST_COUNTRY_NAME STRING,
    ORIGIN_COUNTRY_NAME STRING,
    count LONG
)
USING JSON OPTIONS (path '{json_path_uri}')
""")

URI path: 'file:///c:/ds_analytics_projects/darshil_course/apache-pyspark/darshil-pyspark/data/darshil-data/flight-data/json/2015-summary.json'


DataFrame[]

In [19]:
# Check if table exists and show its structure
spark.sql("DESCRIBE flights").show()

# Show some sample data
spark.sql("SELECT * FROM flights LIMIT 5").show()

# Count rows
spark.sql("SELECT COUNT(*) FROM flights").show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|  DEST_COUNTRY_NAME|   string|   NULL|
|ORIGIN_COUNTRY_NAME|   string|   NULL|
|              count|   bigint|   NULL|
+-------------------+---------+-------+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+

+--------+
|count(1)|
+--------+
|     256|
+--------+



**Create table from a query:**

In [20]:
spark.sql("DROP TABLE IF EXISTS flights_from_select")

spark.sql("""
CREATE TABLE flights_from_select
USING parquet AS
SELECT * FROM flights
""")

DataFrame[]

**Create unmanaged (external) table:**

In [21]:
hive_path = str(Path(get_project_path('data', 'darshil-data', 'flight-data-hive')))
# Convert to file URI
hive_path_uri = Path(hive_path).as_uri()
print(f"URI path: '{hive_path_uri}'")

print(f"Directory: {hive_path}")
print(f"Directory exists: {os.path.exists(hive_path)}")

if os.path.exists(hive_path):
    print("Files in directory:")
    for file in os.listdir(hive_path):
        print(f"  - {file}")

URI path: 'file:///c:/ds_analytics_projects/darshil_course/apache-pyspark/darshil-pyspark/data/darshil-data/flight-data-hive'
Directory: c:\ds_analytics_projects\darshil_course\apache-pyspark\darshil-pyspark\data\darshil-data\flight-data-hive
Directory exists: True
Files in directory:
  - part-00000-tid-4721890993021653500-d8ef7f6b-e6e5-4451-af50-08281422f186-0-c000
  - _committed_4721890993021653500
  - _started_4721890993021653500
  - _SUCCESS


In [22]:
spark.sql("DROP TABLE IF EXISTS hive_flights")

spark.sql(f"""
CREATE EXTERNAL TABLE hive_flights (
    DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)

ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '{hive_path_uri+"/"}'
""")

DataFrame[]

---

### 🔎 Step 6: Inserting into Tables

In [23]:
spark.sql("""
INSERT INTO flights_from_select
SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count
FROM flights
LIMIT 20
""")

DataFrame[]

---

### 🔎 Step 7: Useful SQL Commands

- **Describe a table:**

In [24]:
spark.sql("DESCRIBE TABLE flights_view").show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|  DEST_COUNTRY_NAME|   string|   NULL|
|ORIGIN_COUNTRY_NAME|   string|   NULL|
|              count|   bigint|   NULL|
+-------------------+---------+-------+



- **Show partitions of a table:**

In [26]:
# Example: Create a partitioned table from existing table
spark.sql("""
CREATE TABLE flights_from_select_partitioned
USING PARQUET
PARTITIONED BY (DEST_COUNTRY_NAME)
AS SELECT * FROM flights_view
""")

# Now you can show partitions on the new table
spark.sql("SHOW PARTITIONS flights_from_select_partitioned").show()

+--------------------+
|           partition|
+--------------------+
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
|DEST_COUNTRY_NAME...|
+--------------------+
only showing top 20 rows



- **Refresh table metadata:**

In [27]:
spark.sql("REFRESH TABLE flights_from_select_partitioned")

DataFrame[]

- **Repair partitions (Hive-style):**

In [28]:
spark.sql("MSCK REPAIR TABLE flights_from_select_partitioned")

DataFrame[]

- **Drop a table:**

In [29]:
spark.sql("DROP TABLE flights_from_select_partitioned")

DataFrame[]

---

### 🔎 Step 8: Views

- **View = Saved query (no data stored)**.
- Types:
    - **Session-specific view** (temp).
    - **Database-specific view**.
    - **Global view** (across sessions).

Example:

In [30]:
spark.sql("""
CREATE VIEW just_usa_view AS
SELECT * FROM flights WHERE DEST_COUNTRY_NAME = 'United States'
""")

DataFrame[]

---

### 🔎 Step 9: Databases

- Databases organize tables.
- Default database = `default`.
- Create new database:

In [31]:
spark.sql("CREATE DATABASE some_db")

DataFrame[]

👉 Then use with:

In [32]:
spark.sql("USE some_db")

DataFrame[]

---

### 🔎 Step 10: Why this matters?

- Spark SQL unifies **SQL + DataFrame API**.
- Supports Hive compatibility, external tables, and schema management.
- Tables (managed/unmanaged) and views give flexibility in **data persistence**.

---

✅ **In simple words:**

Spark SQL lets you run SQL on top of DataFrames, store them as tables (managed/unmanaged), and organize them into databases. Views = saved queries, tables = actual data, and Hive compatibility makes migration easier.